In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
import time
import config



#To change
from my_load_data import load_dataset_fn 

In [2]:
###COMMENTS
"""
Accuracy makes softmax of whole tensor

CHANGE DATA PIPELINE

"""

'\nAccuracy makes softmax of whole tensor\n\nCHANGE DATA PIPELINE\n\n'

In [3]:
print (len(os.listdir(config.PIC_SRC_DIR)))

1000


In [7]:
#WRAPPERS FOR MODEL
def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator


class Model(object):
    """
    Model of NN
    """
    #Variable of model
    #accuracy = tf.reduce
    
    
    
    def __init__(self):
        #self.permutation = np.random.permutation(range(len(os.listdir(config.PIC_SRC_DIR))))
        self.keep_prob = config.KEEP_PROB
        self.write_step = config.PRINT_NTH
        self.gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')
        self.training = False ### USE IN FUTURE
        
        
        #remove bellow
        self.data = tf.placeholder(dtype= tf.float32, shape=[None,256,256,3])
        self.taget = tf.placeholder(dtype = tf.float32, shape=[None,1,1,1,1,1])
        self.permutation = np.random.permutation(range(6))
        self.data_iter = 1 
        self.total_epoch = 0
        
        #FUNCTIONS, don not remove
        self.prediction
        self.optimize
        self.loss_op
        
      
    #FUNCTION DEFINITIONS
    def data_pipeline(self):
        """
        loading of TFRecords
        """
        pass
    
    
    #CHANGE LOAD DATA TO DATA_PIPELINE
    @define_scope    
    def load_data(self):
        """
        Serve random data each iteration
        """
        with tf.name_scope('Input'):
            self.data, self.target_class, self.target_loc = (load_dataset_fn(self.permutation[self.data_iter]))
            print(type(self.data))
        
   
    @define_scope
    def prediction(self, if_training=True):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, config.CNN_IN_HEIGHT, config.CNN_IN_WIDTH, config.CNN_IN_CH])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = self.data, 
                             filters = 32,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)
        
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #2 conv layer
        conv2 = tf.layers.conv2d(inputs = pool1, 
                             filters = 64,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)

        #2 pool overal image size reduced totaly by factor of 1/16
        pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")
        
        pool2_flat = tf.reshape(pool2,[-1, config.CNN_IN_HEIGHT*config.CNN_IN_WIDTH*4])

        dense = tf.layers.dense(inputs = pool2_flat,
                            units = 512,
                            activation = tf.nn.relu)

        dense2 = tf.layers.dense(inputs = dense,
                             units = config.CNN_OUTPUT_SIZE,
                             activation = tf.nn.relu)
       
        return tf.nn.softmax(dense2)
    
    
    @define_scope
    def loss_op(self):
        """
        loss
        """
        loss = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.prediction,
                                                                labels = self.taget))
        return loss
    
    
    @define_scope    
    def optimize(self):
        """
        Optimizer of model
        """
        optimizer = tf.train.AdamOptimizer(learning_rate=config.LR)
        
        return optimizer.minimize(self.loss_op)
        

        
    def train(self, sess, init, saver, writer,step):
        """
        Training op for model
        """
        start_time = time.time()
        sess.run(init)
        self.training = True
        total_loss = 0
        num_batches = 0
        try: 
            #train
            loss, _ = sess.run([self.loss_op, self.optimize])
            if (step + 1) % self.write_step == 0:
                print('Loss at step {0}: {1}'.format(step, loss))
            #save
            writer.add_summary(summary, global_step=step)
            step += 1
            total_loss += loss
            num_batches += 1
        except KeyboardInterrupt:
            saver.save(sess, ckpt_dir, step)
            print("keyboard interrupt") 
        saver.save(sess, ckpt_dir, step)
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step
        

In [8]:
total_epoch = 100
def main():
    result_dir = config.RESULT_DIR
    
    #Check for dirs, if not present make them
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    ckpt_dir=os.path.join(result_dir,"ckpt")
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(ckpt_dir, graph=graph)
                
            #Initialization of Model, load all Model functions returning variables
            model = Model()
            
            #Assign Initializer
            init = tf.global_variables_initializer()
            
            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            
            #Loading last checkpoint
            ckpt = tf.train.get_checkpoint_state(result_dir)
            if ckpt and ckpt.model_checkpoint_path:
                #if ckpt found load it and load global step
                saver.restore(sess, ckpt.model_checkpoint_path)
                print("Found checkpoint")
                step = model.get_global_step()   
            else: step = 1
            
                    
            #Training
            print("Starting Training")
           
            for epoch in range(total_epoch):
                step = model.train(sess, init, saver, writer,step)
              
            
        
        
        print("Finnished session")
        #Merge all summaries
        #writer.flush()
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")

In [9]:
tf.reset_default_graph()
if __name__ == '__main__':
     main()

Starting Session
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Starting Training


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,256,256,3]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,256,256,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "D:\Python\envs\tf-cpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Python\envs\tf-cpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Python\envs\tf-cpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Python\envs\tf-cpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Python\envs\tf-cpu\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "D:\Python\envs\tf-cpu\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  Fil